In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Moving to directory where data is stored

In [14]:

cd train-1 

/kaggle/input/train-1


# **Importing the required libraries**

In [2]:
import pandas as pd
import numpy as np
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPool2D,AvgPool2D,BatchNormalization,Activation
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.applications import InceptionV3

# Data flow pipeline with some noise additions (brightness change, zoom) and 20% test data

In [32]:
datagen= ImageDataGenerator(
zoom_range=0.15,
brightness_range = [0.3,1.4],
validation_split=0.20)
train_it=datagen.flow_from_directory('train',class_mode='categorical',batch_size=31,target_size=(240,180),shuffle=True,color_mode='grayscale',subset='training')
test_it= datagen.flow_from_directory('train',class_mode='categorical',batch_size=31,target_size=(240,180),shuffle=True,color_mode='grayscale',subset='validation')

Found 1984 images belonging to 62 classes.
Found 496 images belonging to 62 classes.


# Creating variable learning rate with initial rate= 0.001, decay= 0.001/200 and momentum of 0.9

In [34]:
lr_schedule = tensorflow.keras.optimizers.SGD(
    lr=0.001,
    decay= 0.001/200,
    momentum=0.9)

# Proposed Model Architecture (Main Model)

# **The reference for the structure of the model below is inspired from from VGG-16 model with some variations like addition of batch normalization and dropout layer to avoid overfitting (this was the case when I used the below architecture without the use of batch normalization layer) and improves the training speed. The input size considered here is larger(240x180) than mnist hand written data size as the accuracy was only around 76% (shown after this model) after reducing it to 28x28. The activation function used is relu with softmax function at the last dense layer.**

In [35]:
model=Sequential()
model.add(Conv2D(64, (3, 3), padding="same",input_shape=(240,180,1)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3)))
model.add(Dropout(0.2))
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(250))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(62))
model.add(Activation("softmax"))

# Coding was done on kaggle notebook. Due to limited session time I have to save the model weights(after1st training) and again retrain the model. Each time model was trained for 200 epochs. Below the model weights are being loaded for second time training. The model was trained for 400 epochs (i.e 2 times)**

In [16]:
from tensorflow.keras.models import load_model
model=load_model('../weights-final/IIIT (7).h5')

# Compiling the model

In [37]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer= lr_schedule)


# Saving the weights with max val accuracy

In [38]:
from keras.callbacks import ModelCheckpoint
filepath='../../working/IIIT.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Model training for the 2nd time 

In [39]:
model.fit(train_it,steps_per_epoch=int(1984/31),epochs=200,validation_data=test_it,validation_steps=int(496/31),callbacks=callbacks_list)


Epoch 1/200
64/64 [==============================] - 58s 862ms/step - loss: 4.8412 - accuracy: 0.0226 - val_loss: 5.3828 - val_accuracy: 0.0202

Epoch 00001: val_accuracy improved from -inf to 0.02016, saving model to ../../working/IIIT.h5
Epoch 2/200
64/64 [==============================] - 54s 851ms/step - loss: 3.9769 - accuracy: 0.0607 - val_loss: 3.5815 - val_accuracy: 0.1431

Epoch 00002: val_accuracy improved from 0.02016 to 0.14315, saving model to ../../working/IIIT.h5
Epoch 3/200
64/64 [==============================] - 54s 838ms/step - loss: 3.4049 - accuracy: 0.1448 - val_loss: 2.9208 - val_accuracy: 0.2802

Epoch 00003: val_accuracy improved from 0.14315 to 0.28024, saving model to ../../working/IIIT.h5
Epoch 4/200
64/64 [==============================] - 54s 844ms/step - loss: 2.8748 - accuracy: 0.2731 - val_loss: 2.5803 - val_accuracy: 0.3730

Epoch 00004: val_accuracy improved from 0.28024 to 0.37298, saving model to ../../working/IIIT.h5
Epoch 5/200
64/64 [============

# Max accuracy obtained on val data is 83.669 %

# Model with similar architecture but with different input size

In [8]:
model=Sequential()
model.add(Conv2D(64, (5, 5), padding="same",input_shape=(28,28,3)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(250))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(62))
model.add(Activation("softmax"))

# Compiling with same values of learning rate, decay and momentum

In [9]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer= lr_schedule)

# Model training

In [14]:
model.fit(train_it,steps_per_epoch=int(1984/62),epochs=200,validation_data=test_it,validation_steps=int(496/62),callbacks=callbacks_list)


Epoch 1/200
32/32 [==============================] - 40s 1s/step - loss: 4.8360 - accuracy: 0.0222 - val_loss: 4.3908 - val_accuracy: 0.0323

Epoch 00001: val_accuracy improved from 0.01411 to 0.03226, saving model to ../../working/IIIT.h5
Epoch 2/200
32/32 [==============================] - 40s 1s/step - loss: 4.6259 - accuracy: 0.0307 - val_loss: 4.3036 - val_accuracy: 0.0262

Epoch 00002: val_accuracy did not improve from 0.03226
Epoch 3/200
32/32 [==============================] - 40s 1s/step - loss: 4.4427 - accuracy: 0.0358 - val_loss: 4.1228 - val_accuracy: 0.0524

Epoch 00003: val_accuracy improved from 0.03226 to 0.05242, saving model to ../../working/IIIT.h5
Epoch 4/200
32/32 [==============================] - 39s 1s/step - loss: 4.2118 - accuracy: 0.0494 - val_loss: 3.8616 - val_accuracy: 0.0726

Epoch 00004: val_accuracy improved from 0.05242 to 0.07258, saving model to ../../working/IIIT.h5
Epoch 5/200
32/32 [==============================] - 40s 1s/step - loss: 4.0156 - a

# Max accuracy of 76.210% on val data

# I have used the below model for my referencial purpose (dont consider it as part of project). This was just to check the performance of VGG-16 model which was trained from scratch ( I have not used the pretrained weights). The accuracy on val data is less than compared to our chosen model. (Note: val_accuracy did not improve from 0.83669 denotes the val accuracy of our prev(main) model here.)


In [42]:
from tensorflow.keras.applications import VGG16
model1=Sequential()
model1.add(VGG16(weights=None,include_top=False,input_shape=(240,180,1)))
model1.add(Flatten())
model1.add(Dense(250))
model1.add(Activation("relu"))
model1.add(BatchNormalization())
model1.add(Dropout(0.3))
model1.add(Dense(62))
model1.add(Activation("softmax"))

In [43]:
model1.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer= lr_schedule)
model1.fit(train_it,steps_per_epoch=int(1984/62),epochs=200,validation_data=test_it,validation_steps=int(496/62),callbacks=callbacks_list)


Epoch 1/200
32/32 [==============================] - 33s 949ms/step - loss: 4.4864 - accuracy: 0.0137 - val_loss: 16.4830 - val_accuracy: 0.0121

Epoch 00001: val_accuracy did not improve from 0.83669
Epoch 2/200
32/32 [==============================] - 30s 925ms/step - loss: 4.3228 - accuracy: 0.0101 - val_loss: 5.7447 - val_accuracy: 0.0121

Epoch 00002: val_accuracy did not improve from 0.83669
Epoch 3/200
32/32 [==============================] - 29s 916ms/step - loss: 4.2959 - accuracy: 0.0170 - val_loss: 5.4110 - val_accuracy: 0.0161

Epoch 00003: val_accuracy did not improve from 0.83669
Epoch 4/200
32/32 [==============================] - 29s 909ms/step - loss: 4.2653 - accuracy: 0.0202 - val_loss: 5.7179 - val_accuracy: 0.0081

Epoch 00004: val_accuracy did not improve from 0.83669
Epoch 5/200
32/32 [==============================] - 29s 922ms/step - loss: 4.2073 - accuracy: 0.0127 - val_loss: 5.9367 - val_accuracy: 0.0202

Epoch 00005: val_accuracy did not improve from 0.83669